In [1]:
!pip install datasets pandas torch openpyxl banglanltk bangla-stemmer gensim pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from datasets import Dataset
import numpy as np
import wandb
from bangla_stemmer.stemmer import stemmer
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models

In [3]:
# Load the dataset
df = pd.read_excel("dataset_rv.xlsx")
stopwords_df = pd.read_excel('stopwords_bangla.xlsx')

stopwords_df = stopwords_df[['words']]

# Map string labels to numerical labels
label_map = {"outrage": 0, "despair": 1, "hope": 2}
df["Sentiment"] = df["Sentiment"].map(label_map)

# Rename the column to text
df.rename(columns={'Headline':'text', 'Sentiment':'labels'}, inplace=True)
stopwords_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,words
0,অই
1,অগত্যা
2,অত: পর
3,অতএব
4,অথচ


In [4]:
def english_to_bangla_number_text(number):
    """Converts an English number to Bangla textual representation."""
    bangla_numbers = {
        0: "শূন্য", 1: "এক", 2: "দুই", 3: "তিন", 4: "চার",
        5: "পাঁচ", 6: "ছয়", 7: "সাত", 8: "আট", 9: "নয়",
        10: "দশ", 11: "এগারো", 12: "বারো", 13: "তেরো", 14: "চৌদ্দ",
        15: "পনেরো", 16: "ষোলো", 17: "সতেরো", 18: "আঠারো", 19: "উনিশ",
        20: "বিশ", 21: "একুশ", 22: "বাইশ", 23: "তেইশ", 24: "চব্বিশ",
        25: "পঁচিশ", 26: "ছাব্বিশ", 27: "সাতাশ", 28: "আটাশ", 29: "ঊনত্রিশ",
        30: "ত্রিশ", 31: "একত্রিশ", 32: "বত্রিশ", 33: "তেত্রিশ", 34: "চৌত্রিশ",
        35: "পঁত্রিশ", 36: "ছত্রিশ", 37: "সাঁইত্রিশ", 38: "আটত্রিশ", 39: "ঊনচল্লিশ",
        40: "চল্লিশ", 41: "একচল্লিশ", 42: "বিয়াল্লিশ", 43: "তেতাল্লিশ", 44: "চুয়াল্লিশ",
        45: "পঁইয়াল্লিশ", 46: "ছিয়াল্লিশ", 47: "সাতচল্লিশ", 48: "আটচল্লিশ", 49: "ঊনপঞ্চাশ",
        50: "পঞ্চাশ", 51: "একান্ন", 52: "বাহান্ন", 53: "তিপ্পান্ন", 54: "চুয়ান্ন",
        55: "পঞ্চান্ন", 56: "ছাপ্পান্ন", 57: "সাতান্ন", 58: "আটান্ন", 59: "ঊনষাট",
        60: "ষাট", 61: "একষট্টি", 62: "বাষট্টি", 63: "তেষট্টি", 64: "চৌষট্টি",
        65: "পঁষট্টি", 66: "ছেষট্টি", 67: "সাতষট্টি", 68: "আটষট্টি", 69: "ঊনসত্তর",
        70: "সত্তর", 71: "একাত্তর", 72: "বাহাত্তর", 73: "তিয়াত্তর", 74: "চুয়াত্তর",
        75: "পঁচাত্তর", 76: "ছিয়াত্তর", 77: "সাতাত্তর", 78: "আটাত্তর", 79: "ঊনআশি",
        80: "আশি", 81: "একাশি", 82: "বিরাশি", 83: "তিরাশি", 84: "চুরাশি",
        85: "পঁচাশি", 86: "ছিয়াশি", 87: "সাতাশি", 88: "আটাশি", 89: "ঊননব্বই",
        90: "নব্বই", 91: "একানব্বই", 92: "বিরানব্বই", 93: "তিরানব্বই", 94: "চুরানব্বই",
        95: "পঁচানব্বই", 96: "ছিয়ানব্বই", 97: "সাতানব্বই", 98: "আটানব্বই", 99: "নিরানব্বই",
    }

    def convert_two_digit_number(n):
        """Handles numbers from 0 to 99 based on direct mapping."""
        if n in bangla_numbers:
            return bangla_numbers[n]
        tens = (n // 10) * 10
        units = n % 10
        return f"{bangla_numbers[tens]} {bangla_numbers[units]}" if units > 0 else bangla_numbers[tens]

    if number < 100:
        return convert_two_digit_number(number)
    elif number < 1000:
        hundreds = number // 100
        remainder = number % 100
        if remainder == 0:
            return f"{bangla_numbers[hundreds]} শত"
        return f"{bangla_numbers[hundreds]} শত {convert_two_digit_number(remainder)}"
    elif number < 100000: #Handle numbers from 1000 to 99,999
        parts = []
        if number >= 1000:
            parts.append(f"{english_to_bangla_number_text(number // 1000)} হাজার")
            number %= 1000
        if number > 0:
            if number >= 100:
              parts.append(english_to_bangla_number_text(number))
            else:
              parts.append(convert_two_digit_number(number))  #handles cases less than 100
        return " ".join(parts)
    elif number < 10000000: #handles numbers from 100,000 to 9,999,999
      parts = []
      if number >= 100000:
          parts.append(f"{english_to_bangla_number_text(number // 100000)} লক্ষ")
          number %= 100000
      if number > 0:
          if number >= 1000:
            parts.append(english_to_bangla_number_text(number))
          elif number > 0:
            if number >= 100:
                parts.append(english_to_bangla_number_text(number))
            else:
              parts.append(convert_two_digit_number(number)) #handle numbers less than 100
      return " ".join(parts)
    else: # Handle numbers >= 10,000,000 (Crore)
        parts = []
        if number >= 10000000:
            parts.append(f"{english_to_bangla_number_text(number // 10000000)} কোটি")
            number %= 10000000
        if number > 0:
            parts.append(english_to_bangla_number_text(number))
        return " ".join(parts)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
import re

def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\u2600-\u26FF"          # miscellaneous symbols
                               u"\u2700-\u27BF"          # dingbats
                               u"\u2000-\u206F"          # general punctuations
                               "]+", flags=re.UNICODE)
    english_pattern = re.compile('[a-zA-Z]+', flags=re.I)

    text = emoji_pattern.sub(r'', text)
    text = english_pattern.sub(r'', text)

    return text

def remove_punctuations(my_str):
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct += char

    return no_punct

def convert_numbers_to_bangla(text):
    words = text.split()
    converted_words = []
    for word in words:
        if word.isdigit():  # Check if the word is an integer
            bangla_number = english_to_bangla_number_text(int(word))
            converted_words.append(bangla_number)
        else:
            converted_words.append(word)
    return ' '.join(converted_words)

def preprocessing(text):
    text = replace_strings(text)
    text = convert_numbers_to_bangla(text)  # Convert numbers to Bangla
    text = remove_punctuations(text)
    return text

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# Remove Stopwords
stopwords = set(stopwords_df['words'].tolist())
def stopwordRemoval(text):
    return ' '.join([word for word in text.split() if word not in stopwords])

# stemmer function
def stem_text(x):
  stmr = stemmer.BanglaStemmer()
  words=x.split(' ')
  stm = stmr.stem(words)
  words=(' ').join(stm)
  return words

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df['text'] =df['text'].apply(lambda x: preprocessing(str(x)))
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,labels
0,ঢাকা বিশ্ববিদ্যালয় কোটাবিরোধী আন্দোলনের নেতাকে...,0
1,কোটাবিরোধী আন্দোলনে ঢাকা বিশ্ববিদ্যালয়ের বিএনপ...,2
2,কোটাবিরোধী আন্দোলন আজও জিরো পয়েন্ট অবরোধ করে ...,0
3,সর্বজনীন পেনশন সরকার অনড় আন্দোলন চালিয়ে যাবেন ...,0
4,আজ শনিবার সকাল সাড়ে টার দিকে মহাসড়কের শহর বাইপ...,0


In [8]:
df['text'] =df['text'].apply(lambda x: stopwordRemoval(str(x))) # remove stopwords
df['text'] =df['text'].apply(lambda x: stem_text(str(x))) # stem the text
texts = df['text'].tolist()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Streaming output truncated to the last 5000 lines.
applied fourth rules..
applied first rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied third rules..
applied first rules..
applied fourth rules..
applied first rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied second rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied second rules..
applied third rules..
applied first rules..
applied second rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied fourth rules..
applied fourth rules..
applied first rules..
applied first rules..


In [9]:
# Tokenize
def tokenized_data(sent):
    tokenized_text = sent.split()
    return tokenized_text

texts = df['text'].tolist()
df['text'] = [tokenized_data(sent) for sent in texts]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
#Create dictionary
dictionary = Dictionary(df['text'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.get_topics()):
        print(f"Topic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

def calculate_topic_diversity(model, num_topics):
    """Calculates topic diversity."""
    topic_terms = []
    for topic_id in range(num_topics):
        topic_terms.append(set([word for word in model.show_topic(topic_id, topn=10)])) #10 from top 10 words
    if len(topic_terms) <= 1:
       return 0  #Diversity is 0 if less than 2 topics
    else:
      return len(set.union(*topic_terms)) / (num_topics * 10)

def run_topic_modeling(data, num_topics, description):
    print(f"\n--- Topic Modeling for: {description} ---")

    # Create Corpus (Bag of Words)
    corpus = [dictionary.doc2bow(text) for text in data]

    # LDA
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42)

    # Display topics
    display_topics(lda_model, list(dictionary.token2id.keys()), 10) # Pass dictionary words as features

    # Calculate coherence
    coherence_model_lda = CoherenceModel(model=lda_model, texts=df['text'], dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print(f"Coherence Score: {coherence_lda:.4f}")


    # Calculate Perplexity
    perplexity = lda_model.log_perplexity(corpus)
    print(f"Perplexity: {perplexity:.4f}")

    # Calculate Log-Likelihood
    log_likelihood = lda_model.bound(corpus)
    print(f"Log-Likelihood: {log_likelihood:.4f}")

    #Calculate Topic Diversity
    topic_diversity = calculate_topic_diversity(lda_model, num_topics)
    print(f"Topic Diversity: {topic_diversity:.4f}")

    # Save the model
    model_filename = f"{description.replace(' ', '_')}_lda_model.model"
    lda_model.save(model_filename)
    print(f"LDA model saved to: {model_filename}")

    return lda_model, corpus # return the corpus and model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
data_subset = df['text'] # Full dataset model

num_topics = 10
description = "Full DataFrame"

# run topic modeling
lda_model, corpus = run_topic_modeling(data_subset, num_topics, description)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



--- Topic Modeling for: Full DataFrame ---
Topic 0:
কালোটাকা বাস নির্বাচন বুট নগরবাসী গ্যারেজ বেড়ে রাজনৈতিক কীভাবে তৈরি
Topic 1:
জামায়াত যেভাবে দায় বন্যার্ত গেট সাদা পড়েছি গতকাল খুলে ফারাক্কা
Topic 2:
আন্দোলন আওয়ামী সরক কোটা সরকারি রিমান্ডে যুগ শফিকুর ষড়যন্ত্র বদ
Topic 3:
আন্দোলনে চাঁদা থাকা সম্প ফলক টুর্ক নজরুল অবৈধ দাবির শিক
Topic 4:
চিকিৎসক তদন্ কলেজ মেডিকেল চল মির্জা ফখরুল সাঁইত্রিশ লঙ্ঘন ষড়যন্ত্র
Topic 5:
ফেরার আহত ত্রাণ নিয়ন্ত্রণ ফেনী বারবার অভিজ্ঞতা পথে পুরোদমে অপেক্ষায়
Topic 6:
হত্যা চট্টগ্রামে পুলিশ বাড়া চল আগ টহল না থানা কর্মকর্তা
Topic 7:
চোখে না সিটি ঢাকাসহ সেবা কাউন্সিলর পায় ঢাকা পুলিশ শিক্ষার্থী
Topic 8:
বেঁচে গোপন সেন্টিমি মিনিটে কম একাধিক শর্ অধ্যক্ষ ওঠে ভিডিওতে
Topic 9:
বিএনপি অন্যতম অভিযান তল্লাশি বলেছি বিধান অনেকের সংগ্রহ দশ ফির
Coherence Score: 0.5088
Perplexity: -9.5716
Log-Likelihood: -172883.6007
Topic Diversity: 1.0000
LDA model saved to: Full_DataFrame_lda_model.model


In [13]:
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.corpora import Dictionary

# Load the saved LDA model and corpus
model_filename = "Full_DataFrame_lda_model.model"

lda_model = gensim.models.LdaModel.load(model_filename)

# Load the saved dictionary
# Create dictionary from the preprocessed text from previous cell.
dictionary = Dictionary(df['text'])

# Load the corpus - it can be regenerated from the dataset
corpus = [dictionary.doc2bow(text) for text in df['text']]



# Prepare the visualization
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# Enable notebook mode and display the visualization
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
data_subset = df[df['labels']==0]['text'] # outrage model

num_topics = 10
description = "Sentiment_outrage"

# run topic modeling
lda_model, corpus = run_topic_modeling(data_subset, num_topics, description)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



--- Topic Modeling for: Sentiment_outrage ---
Topic 0:
দাবি বিশ্ববিদ্যালয় শিক্ষার্থী পুলিশ আন্দোলনকারী কোটা সংস্কার বিক্ষোভ সংঘর্ষ সরকারি
Topic 1:
সংঘর্ষ পুলিশ ছাত্র আন্দোলন আন্দোলনকারী বৈষম্যবিরোধী হামলা না সিলেট চালা
Topic 2:
আন্দোলন শিক্ষার্থী দাবি লীগ কোটা আন্দোলনকারী সমন্বয়ক কর্মসূচি না আওয়ামী
Topic 3:
দাবি বিশ্ববিদ্যালয় ঢাকা বিক্ষোভ আন্দোলন আন্দোলনকারী আওয়ামী লীগ এলাকায় মিছিল
Topic 4:
শিক্ষার্থী দাবি অবরোধ সংঘর্ষ কোটা পুলিশ আন ঢাকা ফিরা বিশ্ববিদ্যালয়
Topic 5:
বিক্ষোভ দাবি শিক্ষার্থী পুলিশ বিশ্ববিদ্যালয় সংঘর্ষ কোটা আন্দোলনকারী প্রতিবাদ শেখ
Topic 6:
শিক্ষার্থী পুলিশ আন্দোলনকারী আন্দোলন সংঘর্ষ কোটা দাবি চল সরক ঘটনা
Topic 7:
না শিক্ষার্থী দাবি সংস্কার কোটা কথা পুলিশ দেশ কথায় রাজধানীর
Topic 8:
শিক্ষার্থী কোটা না বিশ্ববিদ্যালয় আন্দোলনে দাবি দাঁড়া কর্মসূচি আন্দোলন রাজশাহী
Topic 9:
শিক্ষার্থী পুলিশ বিক্ষোভ দাবি মিছিল অবরোধ রাজধানীর না কর্মসূচি ওপর
Coherence Score: 0.3954
Perplexity: -8.3703
Log-Likelihood: -28171.5438
Topic Diversity: 1.0000
LDA model saved to: Sentiment_outrage_lda_mo

In [15]:
# Load the saved LDA model and corpus
model_filename = "Sentiment_outrage_lda_model.model"


lda_model = gensim.models.LdaModel.load(model_filename)

# Load the saved dictionary (Assuming you have saved it)
# Create dictionary from the preprocessed text from previous cell.
dictionary = Dictionary(df['text'])

# Load the corpus - it can be regenerated from the dataset
corpus = [dictionary.doc2bow(text) for text in df['text']]



# Prepare the visualization
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# Enable notebook mode and display the visualization
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
data_subset = df[df['labels']==1]['text'] # despair model

num_topics = 10
description = "Sentiment_despair"

# run topic modeling
lda_model, corpus = run_topic_modeling(data_subset, num_topics, description)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



--- Topic Modeling for: Sentiment_despair ---
Topic 0:
বন্যা পুলিশ পরিস্থিতির অবনতি কোটা শেখ ফেনী বন্য আন্দোলন বিশ্ববিদ্যালয়
Topic 1:
নদীর নিহত বাঁধ ছাত্রলীগ বন্যা হামলা গ্রাম গোমতী ধাওয়া বন্যায়
Topic 2:
আন্দোলন না সরক বন্যা কোটা শিক্ষার্থী লীগ সময় পতন দেশের
Topic 3:
না বিশ্ববিদ্যালয় ঢাকা পুলিশ হামলা আন্দোলনকারী কোটা শেখ আওয়ামী ছাত্রলীগ
Topic 4:
ঢাকা না হামলা দেশ নেতাকর্মী আন্দোলনকারী ছাত্র কোটা শেখ শিক্ষার্থী
Topic 5:
পানি শিক্ষার্থী পুলিশ মৃত্যু দাবি না দুজন রাজধানীর আহত আন্দোলনকারী
Topic 6:
পুলিশ ব্যাংক ওপর ঘটনা পানি শিক্ষার্থী আন্দোলন ঘট দখল আলম
Topic 7:
না হামলা ওপর আন্দোলন কোটা টাকা নিহত শিক্ষার্থী দেশের দাবি
Topic 8:
কোটা শিক্ষার্থী সংস্ক না আন্দোলন পানি আন্দোলনে মৃত্যু নিহত আহত
Topic 9:
শিক্ষার্থী লাশ না পুলিশ পরীক্ষা লাখ ছাত্রলীগ বড় গ্রেফত এইচএসসি
Coherence Score: 0.4488
Perplexity: -8.6911
Log-Likelihood: -61871.7887
Topic Diversity: 1.0000
LDA model saved to: Sentiment_despair_lda_model.model


In [17]:
# Load the saved LDA model and corpus
model_filename = "Sentiment_despair_lda_model.model"


lda_model = gensim.models.LdaModel.load(model_filename)

# Load the saved dictionary (Assuming you have saved it)
# Create dictionary from the preprocessed text from previous cell.
dictionary = Dictionary(df['text'])

# Load the corpus - it can be regenerated from the dataset
corpus = [dictionary.doc2bow(text) for text in df['text']]



# Prepare the visualization
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# Enable notebook mode and display the visualization
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
data_subset = df[df['labels']==2]['text'] # hope model

num_topics = 10
description = "Sentiment_hope"

# run topic modeling
lda_model, corpus = run_topic_modeling(data_subset, num_topics, description)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



--- Topic Modeling for: Sentiment_hope ---
Topic 0:
উপদেষ্ বাংলাদেশ আন্দোলন ড পুলিশ ছাত্র দল প্রধান শেখ নিষিদ্ধ
Topic 1:
শিক্ষার্থী উপদেষ্ প্রধান বিরুদ্ধে নির্বাচন কথা আলো অন্তর্বর্তী কার্যক্রম উদ্যোগ
Topic 2:
আন্দোলন উপদেষ্ সরকার শেখ প্রধান অন্তর্বর্তীকালীন না শিক্ষার্থী ছাত্র কোটা
Topic 3:
সরকার না ড ইউনূস উপদেষ্ অন্তর্বর্তীকালীন বাংলাদেশ বিশ্ববিদ্যালয় সিদ্ধান্ত অন্তর্বর্তী
Topic 4:
সরকার অন্তর্বর্তীকালীন ড শেখ উপদেষ্ আন্দোলন না ঢাকা ইউনূস দেশ
Topic 5:
পুলিশ কার্যক্রম সরকার নিয়োগ থাকা কারফিউ ব্যক্তি দেশের বিরুদ্ধে সময়
Topic 6:
পুলিশ শিক্ষার্থী সরক বাংলাদেশ কারফিউ অন্তর্বর্তী চল ব্যাংক বিরুদ্ধে বাংলাদেশের
Topic 7:
শিক্ষার্থী উপদেষ্ না দেশের দেশ অন্তর্বর্তী শেখ প্রধান নিষিদ্ধ কোটা
Topic 8:
শিক্ষার্থী সরক ব্যাংক বাংলাদেশ উপদেষ্ হাত শেখ সময় আন্তর্জাতিক আন্দোলনে
Topic 9:
টাকা পুলিশ আন্দোলনে বিরুদ্ধে বাংলাদেশ বৈষম্যবিরোধী ছাত্র গান স্বাভাবিক চল
Coherence Score: 0.3891
Perplexity: -8.6106
Log-Likelihood: -65243.2461
Topic Diversity: 1.0000
LDA model saved to: Sentiment_hope_lda_model.model

In [19]:
# Load the saved LDA model and corpus
model_filename = "Sentiment_hope_lda_model.model"


lda_model = gensim.models.LdaModel.load(model_filename)

# Load the saved dictionary (Assuming you have saved it)
# Create dictionary from the preprocessed text from previous cell.
dictionary = Dictionary(df['text'])

# Load the corpus - it can be regenerated from the dataset
corpus = [dictionary.doc2bow(text) for text in df['text']]


# Prepare the visualization
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# Enable notebook mode and display the visualization
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
